In [ ]:
import geopandas as gpd
import datetime
import rasterio
import rasterio.warp
import os

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import mysecrets
import fetch_from_cluster
import create_stack
import rsutils.modify_images as modify_images
import rsutils.utils
import crop_and_zip

In [ ]:
eth_s2grids_gdf = gpd.read_file('../../ethiopia/data/ethiopia_s2_grids_esa_stats.geojson')

In [ ]:
S2GRIDS_OF_INTEREST = [
    '17b4c3c', # div 4
    '17b37fc', # div 2
    '164c59c', # corner
    '165bca4', # intersection of tiles with different crs
]

In [ ]:
eth_s2grids_gdf.set_index('id').loc[
    S2GRIDS_OF_INTEREST, 'geometry'
].reset_index().to_file('../data/selected_eth_grids.geojson')

In [ ]:
selected_s2grid_gdf = eth_s2grids_gdf[eth_s2grids_gdf['id'] == S2GRIDS_OF_INTEREST[3]][['id', 'geometry']]

In [ ]:
shapes_gdf = selected_s2grid_gdf
startdate = datetime.datetime(2021, 3, 1)
enddate = datetime.datetime(2021, 4, 1)
sshcreds = mysecrets.SSH_UMD_SASIRAJANN
satellite_folderpath = '../data/cluster_files/satellite'
bands = ['B01', 'B02', 'B03', 'B04', 'B08']
overwrite_catalog = False
njobs = 4

In [ ]:
catalog_filepath = fetch_from_cluster.remotepath_to_localpath(
    remotepath = fetch_from_cluster.FILEPATH_SENTINEL2_CATALOG,
    remote_root_path = fetch_from_cluster.FOLDERPATH_SATELLITE,
    local_root_path = satellite_folderpath,
)

fetch_from_cluster.download_file_from_cluster(
    sshcreds = sshcreds,
    remotepath = fetch_from_cluster.FILEPATH_SENTINEL2_CATALOG,
    download_filepath = catalog_filepath,
    overwrite = overwrite_catalog,
)

catalog_gdf = create_stack.filter_catalog(
    catalog_filepath = catalog_filepath,
    shapes_gdf = shapes_gdf,
    startdate = startdate,
    enddate = enddate,
)

In [ ]:
catalog_filepath

In [ ]:
catalog_gdf.shape

In [ ]:
selected_s2grid_gdf

In [ ]:
band_filepaths_df \
= fetch_from_cluster.download_intersecting_sentinel2_tiles_from_cluster(
    shapes_gdf = shapes_gdf,
    startdate = startdate,
    enddate = enddate,
    sshcreds = sshcreds,
    satellite_folderpath = satellite_folderpath,
    bands = bands,
    overwrite_catalog = overwrite_catalog,
    njobs = njobs,
)

In [ ]:
band_filepaths_df

In [ ]:
catalog_gdf['local_folderpath'][0]

In [ ]:
catalog_gdf['local_folderpath'] = catalog_gdf['local_folderpath'].apply(
    lambda folderpath: create_stack.change_parent_folderpath(
        filepath = folderpath,
        parent_folderpath = fetch_from_cluster.FOLDERPATH_SATELLITE,
        new_parent_folderpath = satellite_folderpath,
    )
)

In [ ]:
local_catalog_filepath = rsutils.utils.modify_filepath(
    filepath = create_stack.change_parent_folderpath(
        filepath = fetch_from_cluster.FILEPATH_SENTINEL2_CATALOG,
        parent_folderpath = fetch_from_cluster.FOLDERPATH_SATELLITE,
        new_parent_folderpath = satellite_folderpath,
    ),
    prefix = 'local_'
)
local_catalog_filepath

In [ ]:
catalog_gdf['local_folderpath'][0]

In [ ]:
catalog_gdf.to_file(local_catalog_filepath)

In [ ]:
# zip_filepath = crop_and_zip.crop_and_zip(
#     shapes_gdf = shapes_gdf,
#     catalog_filepath = local_catalog_filepath,
#     startdate = startdate,
#     enddate = enddate,
#     bands = bands,
#     zip_filepath = '../data/testing_crop_and_zip',
#     satellite_folderpath = satellite_folderpath,
# )

In [ ]:
# zip_filepath

In [ ]:
out_folderpath = '../data/testing_create_stack/resample/'

In [ ]:
rsutils.utils.get_all_files_in_folder('../data', keep_extensions=['.jp2.aux.xml'])

In [ ]:
[  
    os.remove(filepath) for filepath in
    rsutils.utils.get_all_files_in_folder('../data', keep_extensions=['.jp2.aux.xml'])
]

In [ ]:
create_stack.create_stack(
    shapes_gdf = shapes_gdf,
    catalog_filepath = local_catalog_filepath,
    startdate = startdate,
    enddate = enddate,
    bands = bands,
    out_folderpath = '../data/testing_create_stack',
    nodata = 0,
    working_dir = '../data/testing_create_stack',
    njobs = njobs,
    dst_crs = None,
    resampling = rasterio.warp.Resampling.nearest,
    resampling_ref_band = 'B08',
)

In [ ]:
bands, metadata = create_stack.load_stack(folderpath='../data/testing_create_stack')

In [ ]:
bands.shape

In [ ]:
metadata

In [ ]:
def get_shape(filepath):
    with rasterio.open(filepath) as src:
        shape = (src.meta['height'], src.meta['width'])
    return shape

In [ ]:
band_filepaths_df['new_shape'] = band_filepaths_df['filepath'].apply(get_shape)

In [ ]:
band_filepaths_df['new_shape'].value_counts()

In [ ]:
timestamps = band_filepaths_df['timestamp'].unique().tolist()
timestamps.sort()
len(timestamps)

In [ ]:
timestamp_band_filepaths_dict = \
band_filepaths_df.groupby(['timestamp'])[
    ['band', 'filepath']
].apply(
    lambda g: dict(map(tuple, g.values.tolist()))
).to_dict()

In [ ]:
bands

In [ ]:
import numpy as np

In [ ]:
stack = []
meta = None
for timestamp in timestamps:
    band_stack = []
    for band in bands:
        band_filepath = timestamp_band_filepaths_dict[timestamp][band]
        with rasterio.open(band_filepath) as src:
            if meta is None:
                meta = src.meta.copy()
            band_stack.append(src.read())
    band_stack = np.stack(band_stack, axis=-1)
    stack.append(band_stack)
    del band_stack
stack = np.concatenate(stack, axis=0)

In [ ]:
stack.shape

In [ ]:
del meta['driver']

In [ ]:
meta

In [ ]:
band_filepaths_df['shape'] = band_filepaths_df['filepath'].apply(get_shape)

In [ ]:
unique_shapes = band_filepaths_df['shape'].unique()

In [ ]:
# unique_shapes = band_filepaths_df['shape'].value_counts().index

In [ ]:
unique_shapes

In [ ]:
filepaths_to_merge = []

for shape in unique_shapes:
    filepath = band_filepaths_df[
        (band_filepaths_df['band'] == 'B08')
        & (band_filepaths_df['shape'] == shape)
    ]['filepath'].tolist()[0]
    filepaths_to_merge.append(filepath)

In [ ]:
filepaths_to_merge

In [ ]:
import rasterio.merge


out_image, out_transform = rasterio.merge.merge(
    filepaths_to_merge,
    nodata = 0,
)

In [ ]:
out_image.shape

In [ ]:
band_filepaths_df

In [ ]:
band_filepaths_df['timestamp'].unique()

In [ ]:
def get_image_crs(filepath:str):
    with rasterio.open(filepath) as src:
        crs = src.crs
    return str(crs)

In [ ]:
with rasterio.open(band_filepaths_df['filepath'][0]) as src:
    meta = src.meta.copy()

meta

In [ ]:
band_filepaths_df['crs'] = band_filepaths_df['filepath'].apply(get_image_crs)

In [ ]:
max_area_contribution_crs = band_filepaths_df.groupby(by='crs')['area_contribution'].mean().sort_values(ascending=False).index[0]
max_area_contribution_crs

In [ ]:
test_filepath = band_filepaths_df[
    (band_filepaths_df['crs'] == max_area_contribution_crs)
    & (band_filepaths_df['band'] == 'B08')
]['filepath'].tolist()[0]

In [ ]:
stack_name = '165bca4'

working_dir = f'../data/test_stack_creation/{stack_name}'
os.makedirs(working_dir, exist_ok=True)

In [ ]:
modify_images.modify_image(
    src_filepath = test_filepath,
    dst_filepath = '../data/test_out_crop_reproject_crop_samecrs_B08.jp2',
    sequence = [
        (modify_images.crop, dict(shapes_gdf=shapes_gdf, nodata=0)),
        (modify_images.reproject, dict(dst_crs=max_area_contribution_crs)),
        (modify_images.crop, dict(shapes_gdf=shapes_gdf, nodata=0)),
    ]
)

In [ ]:
def change_parent_folderpath(
    filepath:str,
    parent_folderpath:str,
    new_parent_folderpath:str,
):
    filepath = os.path.abspath(filepath)
    parent_folderpath = os.path.abspath(parent_folderpath)
    return os.path.join(
        new_parent_folderpath,
        os.path.relpath(path=filepath,
                        start=parent_folderpath)
    )

In [ ]:
change_parent_folderpath(
    filepath = '../data/1/2/3/4/blah.gif',
    parent_folderpath = '../data/1/2',
    new_parent_folderpath = '../data/a/b/'
)

In [ ]:
cropped_imgs_folderpath = os.path.join(working_dir, 'cropped')
os.makedirs(cropped_imgs_folderpath, exist_ok=True)

In [ ]:
def generate_cropped_filepath(
    src_filepath:str,
    parent_folderpath:str,
    _id:str,
):
    crs = get_image_crs(filepath=src_filepath)
    crs_str = crs.lower().replace(':', '-')
    dst_filepath = rsutils.utils.modify_filepath(
        filepath = src_filepath,
        new_folderpath = os.path.join(
            parent_folderpath, 
            crs_str,
            _id,
        )
    )
    return dst_filepath

In [ ]:
band_filepaths_df['cropped_filepath'] = \
band_filepaths_df.apply(
    lambda row: generate_cropped_filepath(
        src_filepath = row['filepath'],
        parent_folderpath = cropped_imgs_folderpath,
        _id = row['id'],
    ),
    axis = 1
)

In [ ]:
band_filepaths_df

In [ ]:
cropped_successes = create_stack.crop_and_save_images(
    src_filepaths = band_filepaths_df['filepath'],
    dst_filepaths = band_filepaths_df['cropped_filepath'],
    shapes_gdf = shapes_gdf,
)

In [ ]:
if not all(cropped_successes):
    raise ValueError('Not all cropping was a success.')

In [ ]:
max_area_contribution_crs

In [ ]:
indexes_to_reproject = band_filepaths_df[
    band_filepaths_df['crs'] != max_area_contribution_crs 
].index

In [ ]:
indexes_to_reproject

In [ ]:
def generate_reprojected_filepath(
    filepath,
    from_crs,
    to_crs,
    _id,
    parent_folderpath,
):
    return rsutils.utils.modify_filepath(
        filepath = filepath,
        new_folderpath = os.path.join(
            parent_folderpath,
            f"{str(from_crs).lower().replace(':', '-')}_to_{str(to_crs).lower().replace(':', '-')}",
            _id,
        )
    )

In [ ]:
reprojected_folderpath = os.path.join(working_dir, 'reprojected')
os.makedirs(reprojected_folderpath, exist_ok=True)

In [ ]:
band_filepaths_df.loc[
    indexes_to_reproject,
    'reprojected_filepath'
] = band_filepaths_df.apply(
    lambda row: generate_reprojected_filepath(
        filepath = row['cropped_filepath'],
        from_crs = row['crs'],
        to_crs = max_area_contribution_crs,
        _id = row['id'],
        parent_folderpath = reprojected_folderpath,
    ),
    axis = 1,
)

In [ ]:
band_filepaths_df

In [ ]:
reproject_successes = create_stack.reproject_images(
    src_filepaths = band_filepaths_df.loc[indexes_to_reproject, 'cropped_filepath'],
    dst_filepaths = band_filepaths_df.loc[indexes_to_reproject, 'reprojected_filepath'],
    dst_crs = max_area_contribution_crs,
    resampling = rasterio.warp.Resampling.nearest,
)

In [ ]:
create_stack.reproject_image(
    src_filepath = 
)